# Single Agent with RAG

Build:
- A small University corpus
- A FAISS vector store and a Retriever (called via `.invoke()`)
- A Retriever Tool (`@tool`)
- A ReAct agent (`create_react_agent`) + `AgentExecutor`

Learning outcome: the agent **uses the tool** to ground its answer.


## 1) Install dependencies

In [1]:
!pip -q install -U langchain langchain-community langchain-openai faiss-cpu tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


## 2) Set API key (OpenAI)
We use `getpass` so your key isn't printed in the notebook output.

In [2]:
import os, getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")
print("Key set:", "OPENAI_API_KEY" in os.environ)


OpenAI API Key: ··········
Key set: True


## 3) Create a small University corpus
These are the kinds of snippets you later retrieve in RAG.

In [3]:
from langchain_core.documents import Document

docs = [
    Document(page_content="Alice is a PhD student researching neuro-symbolic AI, knowledge graphs, and reasoning systems.",
             metadata={"entity":"Alice","type":"Person"}),
    Document(page_content="Bob is a Professor working on ontology engineering and graph-based AI. He supervises students.",
             metadata={"entity":"Bob","type":"Person"}),
    Document(page_content="EnergyAIGroup focuses on AI for sustainability, energy forecasting, optimisation, and climate modelling.",
             metadata={"entity":"EnergyAIGroup","type":"Group"}),
    Document(page_content="University AI policy: use least-privilege access for LLM tools, prefer read-only database permissions in labs, and validate tool outputs.",
             metadata={"entity":"UniversityPolicy","type":"Policy"}),
]
len(docs)


4

## 4) Build a vector store and a retriever
Key idea:
- embeddings turn text into vectors
- vector store supports similarity search
- retriever is the interface we call via `retriever.invoke(query)`

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# Embeddings model (OpenAI)
emb = OpenAIEmbeddings(model="text-embedding-3-small")

# Chunk the docs (important once docs get longer)
splitter = RecursiveCharacterTextSplitter(chunk_size=220, chunk_overlap=40)
chunks = splitter.split_documents(docs)

# Build a local FAISS vector index
vs = FAISS.from_documents(chunks, emb)

# Create a retriever (LangChain v1 style: call with .invoke())
retriever = vs.as_retriever(search_kwargs={"k": 3})

len(chunks)


4

## 5) Inspect retrieval
Before asking the model, always inspect what context is being retrieved.

In [5]:
q = "Who works on neuro-symbolic reasoning?"
hits = retriever.invoke(q)  # v1 retriever call

for h in hits:
    print("----", h.metadata)
    print(h.page_content)


---- {'entity': 'Alice', 'type': 'Person'}
Alice is a PhD student researching neuro-symbolic AI, knowledge graphs, and reasoning systems.
---- {'entity': 'Bob', 'type': 'Person'}
Bob is a Professor working on ontology engineering and graph-based AI. He supervises students.
---- {'entity': 'UniversityPolicy', 'type': 'Policy'}
University AI policy: use least-privilege access for LLM tools, prefer read-only database permissions in labs, and validate tool outputs.


## 6) Wrap retrieval as a Tool
Tools are functions the agent can call.
We return a short evidence bundle that the agent can cite.

In [6]:
from langchain.tools import tool

@tool
def university_retriever(query: str) -> str:
    """Retrieve relevant university snippets (profiles, groups, policies) for a query."""
    docs = retriever.invoke(query)
    # Return a compact evidence string
    return "\n\n".join([f"[{d.metadata}] {d.page_content}" for d in docs])

tools = [university_retriever]
tools


[StructuredTool(name='university_retriever', description='Retrieve relevant university snippets (profiles, groups, policies) for a query.', args_schema=<class 'langchain_core.utils.pydantic.university_retriever'>, func=<function university_retriever at 0x7aa820b68cc0>)]

## 7) Create a ReAct agent (v1) and an AgentExecutor
LangChain v1 uses:
- `create_react_agent` to define how the model reasons with tools
- `AgentExecutor` to run the loop (think → tool → observe → answer)

In [9]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain.messages import HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

@tool
def university_retriever(query: str) -> str:
    """Retrieve relevant university snippets for a query."""
    docs = retriever.invoke(query)  # retriever is a Runnable in v1
    return "\n\n".join([d.page_content for d in docs])

agent = create_agent(model, tools=[university_retriever])

result = agent.invoke(
    {"messages": [HumanMessage(content="Who works on neuro-symbolic reasoning? Use the tool.")]},
)

print(result)

{'messages': [HumanMessage(content='Who works on neuro-symbolic reasoning? Use the tool.', additional_kwargs={}, response_metadata={}, id='3cec2706-15c3-4a72-b752-e9eaa45327bf'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 59, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'id': 'chatcmpl-CnI7XoXBWKf28A4GYeyzbxrDBmQax', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b25ae-a8e7-7453-b266-228c6ba9595e-0', tool_calls=[{'name': 'university_retriever', 'args': {'query': 'neuro-symbolic reasoning'}, 'id': 'call_HXikjvqEK3UVD8FAKqh1LwEw', 'type': 'tool_call'}], usage_metadata={

## 8) Run the agent

In [12]:
agent.invoke(
    {"messages": [HumanMessage(content="Who works on neuro-symbolic reasoning? Use evidence.")]},
)

{'messages': [HumanMessage(content='Who works on neuro-symbolic reasoning? Use evidence.', additional_kwargs={}, response_metadata={}, id='828d5a76-ecdb-4009-8f8b-108aeea17fae'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 58, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'id': 'chatcmpl-CnI8m8XmKQsm4zJ5kZYpCpGH2RY5A', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b25af-d9af-7d32-9754-86a4315b5ed5-0', tool_calls=[{'name': 'university_retriever', 'args': {'query': 'neuro-symbolic reasoning'}, 'id': 'call_HPV4WRqkCW5Ow1PwuZCH02rE', 'type': 'tool_call'}], usage_metadata

In [13]:
agent.invoke(
    {"messages": [HumanMessage(content="What database permissions are recommended for LLM tools in student labs? Use evidence.")]},
)


{'messages': [HumanMessage(content='What database permissions are recommended for LLM tools in student labs? Use evidence.', additional_kwargs={}, response_metadata={}, id='49e188a0-2919-499a-9dbf-154ce9558d7e'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 63, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'id': 'chatcmpl-CnIEfwkDhXz732MpT4S6urnZRige7', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b25b5-6d3b-7030-a282-318c87438d37-0', tool_calls=[{'name': 'university_retriever', 'args': {'query': 'database permissions for LLM tools in student labs'}, 'id': 'call_0EIn

## Reflection
- What did the tool contribute?
- What did the LLM contribute?
- What happens if you set k=1?
